<a href="https://colab.research.google.com/github/RMoulla/SSL26/blob/main/Copie_TP_CLIP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# TP : apprentissage multimodal


Dans ce TP, nous allons utiliser le modèle d'apprentissage, FashionCLIP, pré-entraîné sur des images ainsi que des descriptions en langage naturel. Plus particulièrement, nous allons considérer deux cas d'usage :

*   **Moteur de recherche d'images :** il s'agit de trouver, à partir d'une requête en langage naturel, l'image correspondante.

*   **Classification zero-shot :** il s'agit simplement de construire un classifieur d'images (faire correspondre un label à une image).



## Dataset

Nous allons dans un premier temps télécharger les données. Celles-ci provienennt de [Kaggle](https://www.kaggle.com/competitions/h-and-m-personalized-fashion-recommendations).

In [ ]:
%%capture
!pip install gdown
!gdown "1igAuIEW_4h_51BG1o05WS0Q0-Cp17_-t&confirm=t"
!unzip data


### Modèle FashionCLIP

Nous allons également télécharger le modèle pré-entraîné.

In [ ]:
%%capture
!pip install -U fashion-clip


In [ ]:
import sys
#sys.path.append("fashion-clip/")
from fashion_clip.fashion_clip import FashionCLIP
import pandas as pd
import numpy as np
from collections import Counter
from PIL import Image
import numpy as np
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split
from sklearn.metrics import *
from sklearn.linear_model import LogisticRegression


In [ ]:
%%capture
fclip = FashionCLIP('fashion-clip')


FashionCLIP, à l'instar de CLIP, crée un espace vectoriel partagé pour les images et le texte. Cela permet de nombreuses applications, telles que la recherche (trouver l'image la plus similaire à une requête donnée) ou la classification zero-shot.

Il y a principalement deux composants : un encodeur d'image (pour générer un vecteur à partir d'une image) et un encodeur de texte (pour générer un vecteur à partir d'un texte).










<img src="https://miro.medium.com/v2/resize:fit:1400/0*FLNMtW6jK51fm7Og"  width="400">



Nous allons télécharger les données que nous allons ensuite nettoyer.

In [ ]:
articles = pd.read_csv("data_for_fashion_clip/articles.csv")

# Supprimer les éléments ayant la même description
subset = articles.drop_duplicates("detail_desc").copy()

# upprimer les images dont la catégrie n'est pas renseignée
subset = subset[~subset["product_group_name"].isin(["Unknown"])]

# Garder seulement les descriptions dont la longueur est inférieure à 40 tokens
subset = subset[subset["detail_desc"].apply(lambda x : 4 < len(str(x).split()) < 40)]

# Supprimer les articles qui ne sont pas suffisamment fréquents dans le jeu de données
most_frequent_product_types = [k for k, v in dict(Counter(subset["product_type_name"].tolist())).items() if v > 10]
subset = subset[subset["product_type_name"].isin(most_frequent_product_types)]

subset.head(3)


In [ ]:
subset.to_csv("subset_data.csv", index=False)
f"Il y a {len(subset)} éléments dans le dataset"


## Moteur de recherche d'images

Constuire un moteur de recherche qui permet, à partir d'une description en langage naturel, de récupérer l'image correspondante. Mesurer ses performances (précision).

<img src="https://miro.medium.com/v2/resize:fit:1400/1*cnKHgLAumVyuHuK9pkqr7A.gif"  width="800">


In [ ]:
images = ["data_for_fashion_clip/" + str(k) + ".jpg" for k in subset["article_id"].tolist()]
texts = subset["detail_desc"].tolist()

image_embeddings = fclip.encode_images(images, batch_size=32)
text_embeddings = fclip.encode_text(texts, batch_size=32)

import matplotlib.pyplot as plt

def search_engine(query, top_k=5):
    query_embedding = fclip.encode_text([query])
    similarities = np.dot(image_embeddings, query_embedding.T).squeeze()
    top_indices = np.argsort(similarities)[::-1][:top_k]
    
    fig, axes = plt.subplots(1, top_k, figsize=(15, 3))
    for i, idx in enumerate(top_indices):
        img = Image.open(images[idx])
        axes[i].imshow(img)
        axes[i].set_title(f"Score: {similarities[idx]:.3f}")
        axes[i].axis('off')
    plt.show()
    return top_indices


In [ ]:
print(image_embeddings.shape)
print(text_embeddings.shape)


In [ ]:
search_engine("a pink dress")


Notre moteur de recherche semble bien fonctionner.

# Classification zero-shot

Construite un classsifieur d'images (prédire le label d'une image). Mesurer ses performances.

<img src="https://miro.medium.com/v2/resize:fit:1400/1*No6ZONpQMIcfFaNMOI5oNw.gif"  width="800">



On utilisera les types de produit comme labels.

In [ ]:
labels = subset["product_type_name"].unique().tolist()
print(f"Nombre de labels: {len(labels)}")
labels[:10]


In [ ]:
label_embeddings = fclip.encode_text(labels)

def zero_shot_classify(image_path):
    img_embedding = fclip.encode_images([image_path])
    similarities = np.dot(img_embedding, label_embeddings.T).squeeze()
    return labels[np.argmax(similarities)]

from tqdm import tqdm

y_true = subset["product_type_name"].tolist()
y_pred = [zero_shot_classify(img) for img in tqdm(images)]

print(classification_report(y_true, y_pred))


On obtient de très bon résultats. On a un score de 0 sur top mais c'est un mot peu précis.

Ci-dessous une démonstration du classifier.

In [ ]:
def demo_classifier(idx):
    image_path = images[idx]
    true_label = y_true[idx]
    pred_label = zero_shot_classify(image_path)
    
    img = Image.open(image_path)
    plt.figure(figsize=(5, 5))
    plt.imshow(img)
    plt.title(f"True: {true_label}\nPredicted: {pred_label}")
    plt.axis('off')
    plt.show()

demo_classifier(0)
demo_classifier(10)
demo_classifier(50)
